In [1]:
# Common stuff

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (9, 6)

In [2]:
# Let's scrap weather site

import urllib.request
from scrapy.selector import Selector

def scrape_weather(url):
    html = urllib.request.urlopen(url)
    html = html.read()
    tm_h = Selector(text=html).xpath('//div[@data-widget-id="forecast"]//div[@class="w_time"]/span/text()').extract()
    temperature = Selector(text=html).xpath('//div[@data-widget-id="forecast"]//div[@class="chart chart__temperature"]/div[@class="values"]/div[@class="value"]/text()').extract()
    temperature = [x.strip().replace('−', '-') for x in temperature]
    prec = Selector(text=html).xpath('//div[@data-widget-id="forecast"]//div[@class="widget__item w_precipitation"]/div/text()').extract()
    prec = [x.strip().replace(',', '.') for x in prec]
    wind = Selector(text=html).xpath('//div[@data-widget-id="forecast"]//div[@class="w_wind"]/div/text()').extract()
    wind = [x.strip() for x in wind]
    
    if not prec:
        # Handle 'no precipitation' case
        prec = [0] * len(tm_h)
    
    res = pd.DataFrame(columns=['tm_h', 'temp', 'prec', 'wind_1', 'wind_2'])
        
    if (len(tm_h) != len(temperature)) or (len(tm_h) != len(prec)) or (len(tm_h) != len(wind)) or (len(tm_h) < 2):
        print("Site layout has changhed :(")
        return res
            
    for i in range(len(tm_h)):
        try:
            tm_h[i] = int(tm_h[i])
        except ValueError:
            print("Cannot convert time :(")
            return res
        try:
            temperature[i] = int(temperature[i])
        except ValueError as e:
            print("Cannot convert temperature :(,", e)
            return res
        try:
            prec[i] = float(prec[i])
        except ValueError as e:
            prec[i] = np.nan
        try:
            tmp = wind[i].split('-')
            if len(tmp) > 1:
                if len(tmp) != 2:
                    print("Cannot convert wind :(")
                    return res
                wind[i] = (int(tmp[0]), int(tmp[1]))
            else:
                wind[i] = (int(wind[i]), int(wind[i]))
        except ValueError as e:
            print("Cannot convert wind :(,", e)
            return res
                
    for i in range(len(tm_h)):
        res = res.append({'tm_h' : tm_h[i], 'temp' : temperature[i], 'prec' : prec[i], 'wind_1' : wind[i][0], 'wind_2' : wind[i][1]}, ignore_index=True)
    
    return res                    
    
weather = scrape_weather("https://www.gismeteo.ru/weather-moscow-4368")
#weather = scrape_weather("https://www.gismeteo.ru/weather-carmen-304919")

if len(weather) == 0:
    assert(False)
    
display(weather)

,tm_h,temp,prec,wind_1,wind_2
0,0,-4,NaN,1,1
1,3,-4,NaN,1,1
2,6,-4,NaN,0,0
3,9,-4,0.0,1,1
4,12,-5,0.0,3,5
5,15,-4,0.1,4,7
6,18,-4,0.0,4,7
7,21,-4,0.0,5,9


In [3]:
# Let's output stuff

import pymorphy2, math
morph = pymorphy2.MorphAnalyzer()

def awn_d(n, s):
    return "%d %s" % (int(n), morph.parse(s)[0].make_agree_with_number(abs(n)).word)

def awn_f(n, s):
    return "%.1f %s" % (n, morph.parse(s)[0].make_agree_with_number(round(math.modf(abs(n))[0] * 10)).word)

for w in weather.itertuples():
    print("Температура в %s составит %s, скорость ветра " % (awn_d(w.tm_h, "час"), awn_d(w.temp, "градус")), end='')    
    if w.wind_1 != w.wind_2:
        print("от %d до %s в секунду" % (w.wind_1, awn_d(w.wind_2, "метр")), end='')
    else:
        print("%s в секунду" % (awn_d(w.wind_1, "метр"),), end='')
    if pd.isnull(w.prec):
        print(", нет данных по осадкам")
    elif w.prec == 0.0:
        print(", без осадков")
    else:
        print(", осадки составят %s" % (awn_f(w.prec, "миллиметров"),))

Температура в 0 часов составит -4 градуса, скорость ветра 1 метр в секунду, нет данных по осадкам
Температура в 3 часа составит -4 градуса, скорость ветра 1 метр в секунду, нет данных по осадкам
Температура в 6 часов составит -4 градуса, скорость ветра 0 метров в секунду, нет данных по осадкам
Температура в 9 часов составит -4 градуса, скорость ветра 1 метр в секунду, без осадков
Температура в 12 часов составит -5 градусов, скорость ветра от 3 до 5 метров в секунду, без осадков
Температура в 15 часов составит -4 градуса, скорость ветра от 4 до 7 метров в секунду, осадки составят 0.1 миллиметра
Температура в 18 часов составит -4 градуса, скорость ветра от 4 до 7 метров в секунду, без осадков
Температура в 21 час составит -4 градуса, скорость ветра от 5 до 9 метров в секунду, без осадков
